<a href="https://colab.research.google.com/github/antonFJohansson/Article_collection/blob/master/Bayesian_leave_one_out_cross_validation_for_large_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-------------------------------
We are interested in selecting a model by the use of Leave-One-Out Cross-Valdiation (LOO-CV). They frame this in a Bayesian decision theoretic framework where we are interested in maximizing the expected utility as [Q1]
$$u(a) = \int u(a, \tilde{y}_i)p(\tilde{y}_i)d\tilde{y}_i$$
We are interested in obtaining a predictive model $M$ and thus we use the log score function ($\log p_M(\tilde{y}|y)$) as our utility function, where $y$ is the data that we have observed [Q2]. From this we can make the expected log predictive density (elpd) as
$$elpd_M = \int \log p_M(\tilde{y}_i|y)p(\tilde{y}_i)d\tilde{y}_i$$

##Questions/Answers
* [Q1] Maybe I would have preferred just writing $y$ in the integral instead of $\tilde{y}_i$. Why the subscript on $i$? 
* [Q2] Why do we condition on $y$ in our utility function? Intuitively it makes sense since we have observed some data and want to find the model that is the best one for the data that we observed. But at the same time it feels like we would like to find the best model even if we had not observed this data? That also have a bar over this quantity in their article but I assume that is a misprint?
* [A1] I guess the conditioning on $i$ is that later we will create a model for $y_{-i}$ and see how it predicts for the missing observation $i$.

-------------------------------
Then we start working with LOO-CV, here we can treat our observed $\tilde{y}_i$ as pseudo-samples. Pseudo here in the sense that we cannot get new observations I presume, but can only resample those we have observed. This gives us an estimate for $elpd_{loo}$ as [Q3]
$$\bar{elpd}_{loo} = \frac{1}{n}\sum \log p_M(y_i|y_{-i}) = \frac{1}{n}\sum \log \int p_M (y_i|\theta)pM(\theta|y_{-i})d\theta = \frac{1}{n}elpd_{loo}$$

They later then mention that regular k-fold CV is a biased estimator of $elpd_M$, this is understandable since no matter how much data we have we will still split away a huge part of it, for LOO-CV the effect of removing one part will be less and less and thus intuitively it can be understood to be consistent (where consistent means that the estimator converges to the true estimate in probability as $n\rightarrow \infty$).

Small sidenote here: LOO-CV is consistent for both regular and singular models. A regular model is one which has a bijective transformation between the parameter space and the corresponding model and the Fisher information is positive definite. A neural network does not need to be regular and is thus singular. It can therefore be good to consider model selection criterias that are consistent for singular models if one wants to find an optimal neural network architecture.

Main problems with LOO-CV:
* We have to train $n$ models whcih can be very time-consuming for large $n$.
* In order to get the value of $elpd$ we need all the contributions of $\log p_M(y_i|y_{-i})$ and if these terms are costly to compute then it can take a long time.
## Questions/Answers
* [Q3] Inuitively I can understand this definition as estimating something for the withheld observation while we trained on the remaining data. But this is not the same definition as we used above where we use the true data generating model. Also why is there a bar in the beginning but not the end? The end part should also be an estimate since we work with finite $n$ right? But is it defined to be this quantity?
* [A3] The first part of $elpd_{loo}$ is given like that since we just see $y_i$ as a pseudo-sample from the true distribution and then that quantity will converge to the $elpd_M$ we define above according to WLLN, or I guess WLLN might not work since we condition on different $y_{-i}$ in each term, but I assume there is some convergence at work here. And I guess in the next steps we are of course free to rewrite the expressions as we desire. A bar in the beginning I can understand since the estimate converges to the true quantity when $n\rightarrow \infty$. 

-------------------------------
We try and approximate these terms using importance sampling since the integral can be very difficult to compute analytically. We get the importance sampling approximation
$$\log \hat{p}(y_i|y_{-i}) = \log \bigg(\frac{\frac{1}{S}\sum_s^Sp_M (y_i|\theta)r(\theta_s)}{\frac{1}{S}\sum_s^Sr(\theta_s)}\bigg)$$
where we draw the samples $\theta_s$ from the full posterior $p(\theta|y)$ and we have the definition [Q4]
$$r(\theta_s) = \frac{p_M(\theta_s|y_{-i})}{p_M(\theta_s|y)} \propto \frac{1}{p_M(y_i|\theta_s)}$$

These ratios $r(\theta_s)$ can be unstable due to long tails (as is common in importance sampling). They resolve this by using Pareto-smoothed Importance sampling.
## Questions/Answers
* [Q4] The definition of $r$ is clear to me but can we even evaluate this in practice since we do not know the normalization constant for the posterior? And do we even need $r$ in the denominator? 
* [A4] I think that they include the case here where $p_M$ is the unnormalized posterior density.

-------------------------------
Pareto-Smoothed Importance sampling.

The Pareto distribution is a distribution with heavy tails that is good to approximate tails. They work with the generalized Pareto-distribution which has the property that if a parameter $k>0$, then the number of existing moments is less than floor($1/k$). The number of moments affect how fast an importance sampling scheme converges so this can yield some important information. The generalized Pareto-distribution is given as
$$p(y|u,\sigma, k)=
\begin{cases}
\frac{1}{\sigma}(1+k(\frac{1-u}{\sigma}))^{-\frac{1}{k}-1}~~k \neq0,\\
\frac{1}{\sigma}\exp (\frac{y-u}{\sigma})~~k=0
\end{cases}$$
Where $k, \sigma$ are parameters and $u$ is a lower bound for the distribution ($y \in (u,\infty)$). In order to estimate these parameters for the distribution they reparametrize the distribution as $(b,k)$ where $b=k/\sigma$. They get these parameters through a series of steps
* We find $k$ first by maximizing the likelihood given $b$ (we assume b is known). This is known as a profile likelihood for $b$.
* This profile likelihood is combined with a weakly informative prior for $b$ and the posterior mean $\hat{b}$ is our estimate for the parameter (they estimate it numerically)(and this combination of likelihood and prior is done just as in the usual Bayesian case).
* The finally, we obtain $\hat{k}$ by maximizing the likelihood given $\hat{b}$ and we put $\hat{\sigma} = \hat{k}/\hat{b}$
* Finally to reduce the variance for small $S$, they add an additional prior for $k$ so the final estimated $k$ is given as $\hat{k} = (M\hat{k}+10 \cdot 0.5)/(M + 10)$ where $M$ is the sample size in the tail. [Q5]
* They choose $u$ such that the amount of samples in the tail $M$ is given by
$$M = \min (S/5, 3\sqrt{S})$$
So their idea here is that we know that importance sampling can have high variance. The usual importance sampling is given as
$$\hat{\mu} = \frac{\frac{1}{S}\sum_s^Sh(\theta_s)r(\theta_s)}{\frac{1}{S}\sum_s^Sr(\theta_s)}$$
Due to the high variance we can replace the weights $r(\theta_s)$ with a function $w(\theta_s)$ where $w\propto 1$ or $w \propto r$ for two extreme cases.  One earlier solution was to truncate these weights so that $ws = \min (r_s, \sqrt{S}\bar{r})$ but this yield biased estimates. The solution thus here is to instead of just truncating the values, we use the Pareto-distribution to remove the high variance weights.

So the method proceeds as follows:
* Perform usual IS to get importance weights $r_s,s=1,...,S$
* Set $M$ = $\min (S/5, 3\sqrt{S})$ and set $u$ accordingly.
* Fir the generalized Pareto distribution to the top $M$ as
$$r_{z} = F^{-1}\big(\frac{z-1}{M}\big),~~z=1,...,M,$$
where the parmeters of the Pareto distribution has been estimated with the likelihood method described above.
* If any of the weights are larger than the largest regular IS sample, then truncate those values to the largest regular IS sample.
* If $\hat{k}>0.7$, report a warning that the samples might be unstable
The last point follows since tonly the first moment exists for such $k$ parameters, ideally we want it as low as possible to have as many moments as possible.

## Questions/Answers
* [Q5] How do they determine which samples lie in the tail here?
* [A5] This is determined by where we choose $u$, the samples in ($u,\infty$) are in the tail.

-------------------------------
The main idea now is to approximate the true posterior with a variational family or perform a Laplace approximation. They will consider a Laplace approximation, a variational approximation with a multivariate normal family with either a diagonal covariance matrix (mean-field) or a full rank covariance matric (full-rank). Thus they will use the posterior approximation as the proposal distribution in the importance sampling scheme. Our new weights are given as [Q6][Q7]
$$r(\theta_s) = \frac{p_M(\theta_s|y_{-i})}{q_M(\theta_s|y)} = \frac{p_M(\theta_s|y_{-i})}{p_M(\theta_s|y)}\frac{p_M(\theta_s|y)}{q_M(\theta_s|y)} \propto \frac{1}{p_M(y_i|\theta_s)} \frac{p_M(\theta_s|y)}{q_M(\theta_s|y)}$$
So using this procedure we can approximate the term
$$\log \hat{p}(y_i|y_{-i}) = \log \bigg(\frac{\frac{1}{S}\sum_s^Sp_M (y_i|\theta)r(\theta_s)}{\frac{1}{S}\sum_s^Sr(\theta_s)}\bigg)$$
and then sum it to get an estimate of $elpd_{loo}$ which we can use to select which model to use. Since these terms might be many, the main bottleneck of the algorithm can be just estimating all these terms. Their suggestions is then to use a sample to estimate the sum. They write "Estimating totals, such as $elpd_{loo}$, has a long tradition in sampling theory" but what do they mean with totals here? 

In order to estimate this in an easy way they use some tricks.
* The Hansen-Hurwitz estimator: This estimator can be used when one wants to estimate a total (a sum I think) by subsampling a population, where what we do is that we weigh the terms we want to sample by a variable that is assumed to be correlated with the variable of interest. Maybe like sampling different states and we are interested in the number of drunk vehicle accidents, and we then construct the sampling probabilities by using the amount of drivers in each state which should be correlated with this number. If we had just sampled the states at random then the estimate could have higher variances since we might sample the low states too often or the high states too ofte, but by weighing the states we get a more stable estimate. Thus the Hurwitz-Hansen estimator is given as
$$\hat{\tau} = \frac{1}{m}\sum_{i=1}^m\frac{y_i}{p_i}$$
And in our case we get the estimator as
$$ \hat{\bar{elpd_{loo}}} =\frac{1}{nm}\sum_{i=1}^m \frac{\log \hat{p}(y_i|y_{-i})}{\hat{\pi_i}}$$
[Q8], where $\hat{\pi_i}$ is the probability of subsampling observation $i$. Doing this is called probability-proportional-to-size sampling scheme. We can also get the variance of the estimate (which can help us when deciding on which model to select[The variance might actually be something good to have when selecting which model to work with in these cases, so maybe try and get an estimator that has the ability to easily get the variance for]) as
$$Var( \hat{\bar{elpd_{loo}}}) = \frac{1}{n^2m(m-1)}\sum_{i=1}^m\bigg(\frac{\log \hat{p}(y_i|y_{-1})}{\hat{\pi_i}} - n \hat{\bar{elpd_{loo}}}\bigg)^2$$
They argue that there are some benefits to using this estimator such that they can use MCMC schemes and do not completely rely on VI. There is also a scheme called Walker-Alias multionomial sampling which works here where we build a table that we can sample from, we can then sample from this table until our variance estimates are low enough for our purposes of model selection.
## Questions/Answers
* [Q6]: Why do we use propto, will constants go away in the sampling scheme?
* [Q7]: So I assume that we somehow train the variational approximation first, and then use it to get the LOOCV estimates?
* [Q8]: Why do we divide by $n$ here? It seems to me that we should only divide by $m$?
* [A8]: I think when we define the quantity we have a division by $n$ in it.

--------------------------------------
Under some assumptions they show that a wide variety of ways to obtain the subsampling probabilties are valid and that our estimate of $elpd_{loo}$ will converge in probability. The main takeaway from these assumptions might be that the posterior approximation needs to be consistent (I think this entails that the approximation should converge to the true posterior in the limit. But the Laplace approx and mean-field are not consistent I think but they still use them?)

To summarize their method they write as follows:
* Estimate the model using any consistent posterior approximation (I guess this is to make it easier to actually sample from the posterior distribution later)
* They write use $\hat{k}$ to diagnose the posterior but we will only use Pareto importance sampling later?
* Compute the subsample weights $\hat{pi_i} \propto -\log p(y_i|y)$ (this can be approximated with $-\log p(y_i|\hat{\theta})$ for regular models[Q9]).
* Sample $m$ observations and then compute $\log p(y_i|y_{-i})$ by using PIS, use $\hat{k}$ to diagnose the approximation.
* Estimate $\hat{\bar{elpd_{loo}}}$ and its variance and perform model selection based on this information.
* If higher accuracy is needed, repeat step 4 and 5 until a sufficiently low variance has been obtained.
Important that the $\hat{k}$ diagnostic can be too conservative and say that the model is faulty even when it is working, this could be due to a correlated posterior.
## Questions/Answers
* [Q9]: What is a regular model?

-----------------------------------------------------
Could be interesting to apply this method for Neural Networks and see how they work?

So the steps for implementing this would be the same as the one above with the difference of actually fitting the variational distribution to the posterior which might be more difficult I guess.
